In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
pd.set_option('display.max_columns',50)

In [3]:
dataFile = './data.csv'

In [4]:
excludedCols = ['Responsável','Guia','Folder','Artigo','Família.1', 
                'Glossário','Fotos adultos','Fotos girinos',
                'Vídeos','Áudios', 'Diagnose' ]

df = pd.read_csv(dataFile, usecols=lambda x: x not in excludedCols)

In [5]:
df.drop('Red List.1', axis=1, inplace=True)

In [6]:
df.rename( {'Espécie': 'speciesName', 
            'Família': 'family',
            'Postura dos ovos (habitat breeding)*': 'habitat_breeding',
            'Chances de encontros (muito raro/raro/frequente/muito frequente)': 'detectability',
            'Nome Comum': 'vernacularNames'}, axis=1, inplace=True)

In [7]:
df.replace('^-$', '', inplace=True, regex=True)

### Species name, id and Authorship

In [8]:
df['scientificName'] = df['speciesName'].apply(lambda x: re.findall( '^\w+(?: cf.| aff.)? \w+\.?', str(x))[0])

In [9]:
df['scientificNameAuthorship'] = df[['speciesName','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)
df['scientificNameAuthorship'] = df['scientificNameAuthorship'].apply(lambda x: re.findall( '[\w,\s\-&]+', str(x) )).apply(lambda x: ''.join(x).strip())

In [10]:
ids = df['scientificName'].str.replace('\s','-').str.replace('[.]','').str.lower()
df.insert(0, 'id', ids)

In [11]:
df.drop('speciesName', axis=1, inplace=True)

### Family name and id

In [12]:
df['family'] = df['family'].apply(lambda x: x.capitalize())
df['family_id'] = df['family'].str.lower()

### Vernacular name

In [13]:
def extractVernacularNames(string):
    names = re.split('/|;|,',string)
    names = [ n.strip().replace(' ','-').capitalize() for n in names ]
    if len(names)==1 and names[0]=='':
        return []
    else:
        return names

In [14]:
df['vernacularNames'] = df['vernacularNames'].apply(extractVernacularNames)

### Red list

In [15]:
df['Red List'] = df['Red List'].str.lower()
df.loc[ df['Red List']=='não avaliada','Red List']='ne'

In [16]:
df = pd.get_dummies(data=df, prefix='redlist', columns=['Red List'])

### Endemicidade

In [17]:
df['endemic_cerrado'] = df['Endêmico (Cerrado)'].apply(lambda x: 1 if x=='sim' else 0)
df['endemic_chapada'] = df['Endêmico (Chapada)'].apply(lambda x: 1 if x=='sim' else 0)
df.drop('Endêmico (Chapada)', axis=1, inplace=True)
df.drop('Endêmico (Cerrado)', axis=1, inplace=True)

In [18]:
# Helpers

getInsideParentheses = lambda x: [ str.lower(e) for e in re.findall( '\((.{0,5})\)', str(x) ) ]

### Detectability

In [19]:
df['detectability'] = df['detectability']\
    .str.replace('[^\w]','')\
    .str.replace('ê','e')\
    .str.lower()

In [20]:
subst_dict = {
    'muitofrequente': 'ff',
    'frequente':'f',
    'raro': 'r',
    'muitoraro': 'rr'
}

df['detectability'] = df['detectability'].apply(lambda x: subst_dict[x] if x is not np.NaN else x)
df = pd.get_dummies(df, prefix='detectability',columns=['detectability'])

### Poleiro

In [21]:
df['poleiro'] = df['Poleiro (tipical calling perch)* '].apply( getInsideParentheses )
df.drop('Poleiro (tipical calling perch)* ', axis=1, inplace=True)

In [22]:
poleiro_types = list(set( el for ls in df['poleiro'] for el in ls))

In [23]:
for tp in poleiro_types:
    df[f'tcp_{tp}'] = df['poleiro'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [24]:
df.drop('poleiro', axis=1, inplace=True)

### Habitat breeding

In [25]:
matches = df['habitat_breeding'].str.lower().str.extractall('\((\w+)\)')
hb_dummies = pd.get_dummies(matches, prefix="habitat_breeding").groupby(level=0).sum()

In [26]:
hb_dummies = hb_dummies.reindex(df.index).fillna(0).astype(int)

In [27]:
df = pd.concat([df,hb_dummies], axis=1)

In [28]:
df.drop('habitat_breeding',axis=1, inplace=True)

### Habitat

In [29]:
df['habitat'] = df['Habitat de vida (mata ou floresta/area aberta/pedras)*'].apply( getInsideParentheses)
df.drop('Habitat de vida (mata ou floresta/area aberta/pedras)*', axis=1, inplace=True)

In [30]:
habitat_types = list(set( el for ls in df['habitat'] for el in ls ))

In [31]:
for tp in habitat_types:
    df[f'habitat_{tp}'] = df['habitat'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [32]:
df.drop('habitat', axis=1, inplace=True)

### Atividade

In [33]:
df['Atividade (noturno/diurno)'].value_counts()

noturno              41
noturno e diurno      4
diurno                3
noturno e diurno      1
Name: Atividade (noturno/diurno), dtype: int64

In [34]:
df['atividade_diu'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'diurno' in str(x) else 0)
df['atividade_not'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'noturno' in str(x) else 0)
df.drop('Atividade (noturno/diurno)', axis=1, inplace=True)

In [35]:
df['Ameaças - SITE REUBER (espécies do DF)'][5]

'Desmatamento e remoção de hábitats constituem ameaças as populações locais.'

### Tamanho

In [36]:
df['tamanho_femea'] = df['Tamanho real (mm) - fêmea'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))
df['tamanho_macho'] = df['Tamanho real (mm) - macho'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))

In [37]:
df['tamanho_femea_min'] = df['tamanho_femea'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
df['tamanho_femea_max'] = df['tamanho_femea'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)
df['tamanho_macho_min'] = df['tamanho_macho'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
df['tamanho_macho_max'] = df['tamanho_macho'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)

In [38]:
df.drop('tamanho_femea', axis=1, inplace=True)
df.drop('tamanho_macho', axis=1, inplace=True)
df.drop('Tamanho real (mm) - fêmea', axis=1, inplace=True)
df.drop('Tamanho real (mm) - macho', axis=1, inplace=True)

In [39]:
df['tamanho_macho_max'].replace(np.NaN, df['tamanho_femea_max'], inplace=True)
df['tamanho_macho_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)
df['tamanho_femea_max'].replace(np.NaN, df['tamanho_macho_max'], inplace=True)
df['tamanho_femea_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)

### Temporarily remove fields

In [40]:
df.drop('Distribuição - SITE REUBER (espécies do DF)', axis=1, inplace=True)
df.drop('Ameaças - SITE REUBER (espécies do DF)', axis=1, inplace=True)

In [43]:
df[['scientificName','tamanho_femea_max']]

,scientificName,tamanho_femea_max
0,Allobates goianus,17.0
1,Rhaebo guttatus,180.0
2,Rhinella mirandaribeiroi,73.0
3,Rhinella rubescens,NaN
4,Rhinella schneideri,200.0
5,Barycholos ternetzi,29.4
6,Ameerega flavopicta,24.0
7,Aplastodiscus lutzorum,NaN
8,Bokermannohyla pseudopseudis,61.6
9,Dendropsophus jimi,NaN


### Write to json

In [51]:
d = df.to_dict(orient='records')

with open('datajson.json', 'w') as f:
    json.dump(d, f,indent=1, ensure_ascii=False)

In [61]:
with open('datajson.json', 'r') as f:
    spdata = json.load(f)

In [62]:
[ i['id'] for i in spdata ]

['allobates-goianus',
 'rhaebo-guttatus',
 'rhinella-mirandaribeiroi',
 'rhinella-rubescens',
 'rhinella-schneideri',
 'barycholos-ternetzi',
 'ameerega-flavopicta',
 'aplastodiscus-lutzorum',
 'bokermannohyla-pseudopseudis',
 'dendropsophus-jimi',
 'dendropsophus-minutus',
 'dendropsophus-rubicundulus',
 'boana-albopunctatus',
 'boana-ericae',
 'boana-goianus',
 'boana-lundii',
 'boana-phaeopleura',
 'boana-raniceps',
 'boana-sp',
 'pithecopus-hypochondrialis',
 'pithecopus-oreades',
 'scinax-aff-squalirostris',
 'scinax-fuscomarginatus',
 'scinax-fuscovarius',
 'scinax-rogerioi',
 'scinax-similis',
 'scinax-skaios',
 'scinax-tigrinus',
 'scinax-rupestris',
 'trachycephalus-typhonius',
 'adenomera-cotuba',
 'adenomera-juikitam',
 'adenomera-saci',
 'leptodactylus-furnarius',
 'leptodactylus-fuscus',
 'leptodactylus-labyrinthicus',
 'leptodactylus-mystaceus',
 'leptodactylus-mystacinus',
 'leptodactylus-sertanejo',
 'leptodatcylus-syphax',
 'leptodactylus-tapiti',
 'leptodactylus-latra

In [54]:
json.loads('datajson.json')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)